In [1]:
import pandas as pd
import json

import bokeh.plotting as bpl
import os
import numpy as np
import math
from bokeh.plotting import figure, output_file, show, gridplot
from bokeh.models import ColumnDataSource, LabelSet, HoverTool, Div, Label, CustomJS
from bokeh.models.widgets import Panel, Tabs
import re

In [2]:
os.chdir('../')
with open('data/raw/cohort_demographics_test_data.json', 'r') as fb:
    cohorts_dict = json.load(fb)
with open('data/raw/Reference_population.json', 'r') as fb:
    reference_dict = json.load(fb)
    
import represetnation_labels.useful_functions as uf 

# Format data

In [3]:
graph_dict = {}

for dataset, variables in cohorts_dict.items():
    graph_dict[dataset] = {}
    for var, vals in cohorts_dict[dataset].items():
        if var == 'Ethnicity':
            graph_dict[dataset][var] = uf.multi_eth_change_format(vals)
        elif isinstance(list(vals.values())[0],dict):
            graph_dict[dataset][var] = uf.multi_change_format(vals,var)
        else:
            graph_dict[dataset][var] = uf.change_format(vals, var)

ref_dict = {}
for var,vals in reference_dict['2011 Census'].items():
    if var == 'Ethnicity':
        ref_dict[var] = uf.eth_change_format(vals,'ethnicity')
    else:
        ref_dict[var] = uf.change_format(vals, var)
        
for var,vals in ref_dict.items():
    ref_dict[var]['percent'] = uf.get_percents(list(vals['values']))

for dataset, variables in graph_dict.items():
    for var,vals in variables.items():
        missing = {uf.get_name(k,'missing'):[v[-1]]*(len(v) -1) for k,v in vals.items() if isinstance(v[0],int)}
        perc_dict = {uf.get_name(k,'percent'): uf.get_percents(v) for k,v in vals.items() if isinstance(v[0],int)}
        ref_pers = ref_dict[var]['percent']
        rel_dict = {uf.get_name(k,'reletive'): uf.get_reletives(v,ref_pers) for k,v in perc_dict.items()}
        std_ref_dict = {uf.get_name(k,'reference standardised'):
                            uf.standardise_refs(v,ref_pers) for k,v in vals.items() if isinstance(v[0],int)}
        vals_short = {k:v[:-1] for k,v in vals.items()}
        ref_pers = {'ref percent': ref_pers}
        graph_dict[dataset][var]={**vals_short,**perc_dict,**rel_dict,**std_ref_dict,**missing,**ref_pers}

In [4]:
print(graph_dict)
print(ref_dict)
print(graph_dict.keys())

{'UK Biobank': {'Age': {'Age': ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-70', '70-74', '75-79', '80-84', '85+'], 'values': [0, 0, 0, 0, 0, 0, 0, 0, 53960, 66438, 76808, 91953, 121419, 92739, 0, 0, 0, 0], 'percent': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.7, 13.2, 15.3, 18.3, 24.1, 18.4, 0.0, 0.0, 0.0, 0.0], 'reletive': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 146.6, 180.8, 235.4, 321.1, 401.7, 383.3, 0.0, 0.0, 0.0, 0.0], 'reference standardised': [31206, 28186, 29192, 31709, 34226, 34226, 33219, 33219, 36742, 36742, 32716, 28689, 30199, 24159, 19126, 16106, 12080, 11073], 'missing': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ref percent': [6.2, 5.6, 5.8, 6.3, 6.8, 6.8, 6.6, 6.6, 7.3, 7.3, 6.5, 5.7, 6.0, 4.8, 3.8, 3.2, 2.4, 2.2]}, 'Gender': {'Gender': ['Female', 'Male'], 'values': [273824, 229486], 'percent': [54.4, 45.6], 'reletive': [106.9, 92.9], 'reference standardised': [256185, 247125], '

In [5]:
def missing_plot(source,missing_vals,values):
    u = figure(plot_height = 50)
    missing_val = source.data[missing_vals][0]
    val_sum = sum(source.data[values])

    if source.data[missing_vals][0] == 0:
        text = 'There is no missing data in this variable'
    else:
        missing_percent = round(missing_val/(val_sum + missing_val)*100,1)
        text = 'There is ' + str(missing_percent) + ' % missing data in this variable'


    u.patch(x = [0,0,val_sum,val_sum],y = [0,2,2,0], color = 'blue',alpha = 0.35,line_alpha = 0)
    u.patch(x = [val_sum,val_sum,val_sum + missing_val,val_sum + missing_val],y = [0,2,2,0], color = '#a0a0a0',alpha = 0.35,line_alpha = 0)


    citation = Label(x=27, y=5, x_units='screen', y_units='screen',
                     text=text, render_mode='canvas',
                     border_line_alpha=0,
                     background_fill_alpha=0,
                     text_font = 'helvetica', 
                     text_color = '#a0a0a0', text_font_size = '21pt'
                    )
    u.add_layout(citation)

    u.yaxis.major_label_text_font_size = '0pt' 
    u.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    u.yaxis.minor_tick_line_color = None
    u.yaxis.axis_line_color = None
    u.xaxis.major_label_text_font_size = '0pt'
    u.xaxis.major_tick_line_color = None
    u.xaxis.minor_tick_line_color = None
    u.xaxis.axis_line_color = None
    u.xgrid.grid_line_color = None
    u.ygrid.grid_line_color = None
    u.outline_line_width = 0
    return(u)


In [6]:
name = 'HES'

## Testing Age

In [7]:
test_age = graph_dict[name]['Age']
print(test_age.keys())
print(test_age)

dict_keys(['Age', 'values', 'percent', 'reletive', 'reference standardised', 'missing', 'ref percent'])
{'Age': ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85+'], 'values': [182104, 234134, 286164, 338194, 351201, 364209, 377216, 416238, 468268, 364209, 338194, 286164, 273156, 260149, 247142, 208119, 156089, 52030], 'percent': [3.5, 4.5, 5.5, 6.5, 6.7, 7.0, 7.2, 8.0, 9.0, 7.0, 6.5, 5.5, 5.2, 5.0, 4.8, 4.0, 3.0, 1.0], 'reletive': [56.5, 80.4, 94.8, 103.2, 98.5, 102.9, 109.1, 121.2, 123.3, 95.9, 100.0, 96.5, 86.7, 104.2, 126.3, 125.0, 125.0, 45.5], 'reference standardised': [322585, 291367, 301773, 327788, 353803, 353803, 343397, 343397, 379818, 379818, 338194, 296570, 312179, 249743, 197713, 166495, 124872, 114466], 'missing': [102720, 102720, 102720, 102720, 102720, 102720, 102720, 102720, 102720, 102720, 102720, 102720, 102720, 102720, 102720, 102720, 102720, 102720], 'ref percen

In [8]:
output_file('plots/test_age.html')
source = ColumnDataSource(data = test_age)

def age_plot(source,addition,cohort_name):
    values = addition + 'values'
    ref_std = addition + 'reference standardised'
    missing_vals = addition + 'missing'
    percent = addition + 'percent'


    p = figure(
    x_range = list(source.data['Age']), 
    title = 'Age', 
    y_range=(0,max(source.data[values]) * 1.1 )
    )

    p.vbar(
    x = 'Age',
    top = values, 
    width = 0.9, 
    color = 'blue',
    alpha = 0.35,
    legend_label = cohort_name,
    line_alpha = 0,
    source = source
    )

    p.vbar(
    x = 'Age',
    top = ref_std, 
    width = 0.8,
    fill_alpha = 0,
    line_color = '#a0a0a0', 
    line_width = 4,
    line_alpha = 1,
    legend_label = 'UK Population Ratio',
    source = source
    )



    hover2 = HoverTool(tooltips = [
    ('Age range', '@Age'),
    ('Raw values', "@{"+values+"}"),
    ('Percent/%', "@{"+percent+"}{0.0}"),
    ('UK population percent/%', '@{ref percent}{0.0}')
    ],
           mode = 'mouse', name= 'data plot')

    p.yaxis.major_label_text_font_size = '0pt' 
    p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    p.yaxis.minor_tick_line_color = None
    p.yaxis.axis_line_color = None
    p.xaxis.axis_line_color = None
    p.xaxis.major_label_orientation = math.pi/2
    p.xaxis.major_tick_line_color = 'grey'
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.outline_line_width = 0
    p.background_fill_color = '#f5f5f5'
    p.background_fill_alpha = 0.9
    p.legend.location = 'top_left'
    p.title.text_color = '#a0a0a0'
    p.title.text_font_size = '24pt'
    p.title.text_font = "helvetica"
    p.legend.label_text_font = "helvetica"
    p.legend.label_text_color = "#a0a0a0"
    p.add_tools(hover2)

    a = missing_plot(source,missing_vals,values)
    final_plot = gridplot([[p],[a]])
 
    return(final_plot)

if 'values' in source.data.keys():
    age_p = age_plot(source,'',name)
else:
    var_list = [re.sub('values','',i) for i in source.data.keys() if 'values' in i]
    tab_list = [Panel(child=age_plot(source,i,name), title=re.sub(' ','',i)) for i in var_list]
    age_p = Tabs(tabs = tab_list)

## Testing Ethnicity

In [9]:
test_eth = graph_dict[name]['Ethnicity']
print(test_eth)

{'Ethnicity': ['White', 'Black', 'Asian', 'Mixed', 'Other'], 'values': [11971598, 576944, 1442361, 144236, 288472], 'percent': [83.0, 4.0, 10.0, 1.0, 2.0], 'reletive': [93.8, 129.0, 185.2, 47.6, 0], 'reference standardised': [12764896, 447132, 778875, 302896, 129812], 'missing': [430852, 430852, 430852, 430852, 430852], 'ref percent': [88.5, 3.1, 5.4, 2.1, 0.9]}


In [10]:
def ethnicity_plot(data_dict,addition,name):
    
    imp_keys = ['Ethnicity','ref percent'] + [k for k in data_dict.keys() if addition in k]
    
    test_spider = {re.sub(addition,'',k):v for k,v in data_dict.items() if k in imp_keys}
    
    num_vars = len(test_spider['values'])
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    # rotate theta such that the first axis is at the top
    theta += np.pi/2

    def unit_poly_verts(theta):
        """Return vertices of polygon for subplot axes.
        This polygon is circumscribed by a unit circle centered at (0.5, 0.5)
        """
        x0, y0, r = [0.5] * 3
        verts = [(r*np.cos(t) + x0, r*np.sin(t) + y0) for t in theta]
        return verts

    def radar_patch(r, theta):
        yt = (r + 0.01) * np.sin(theta) + 0.5
        xt = (r + 0.01) * np.cos(theta) + 0.5
        return xt, yt

    verts = unit_poly_verts(theta)
    x = [i[0] for i in verts]
    y = [i[1] for i in verts]



    values = np.array(test_spider['values'])
    ref_std = np.array(test_spider['reference standardised'])

    values = values/(sum(values) * 2 )
    ref_std = ref_std/(sum(ref_std) * 2 )

    x_val,y_val = radar_patch(values, theta)
    x_ref,y_ref = radar_patch(ref_std,theta)

    label_eth = test_spider['Ethnicity']

    x_lines = [[0.5,i] for i in x]
    y_lines = [[0.5,i] for i in y]

    new_line_max = np.array([max(values + ref_std)*0.55]*len(values))
    new_x,new_y = radar_patch(new_line_max,theta)
    new_x_lines = [[0.5,i] for i in new_x]
    new_y_lines = [[0.5,i] for i in new_y]

    q = figure(title = 'Ethnicity',x_range=(min(new_x)-0.1,max(new_x)+0.1),y_range=(min(new_y)*0.9,max(new_y)*1.1))

    source = ColumnDataSource(data=dict(x_vals=x_val,
                                        y_vals=y_val,
                                        x_ref=x_ref,
                                        y_ref=y_ref,
                                        x_lines=new_x_lines,
                                        y_lines=new_y_lines,
                                        label_eth = label_eth,
                                        labs_x_cords = x,
                                        labs_y_cords = y,
                                        values = test_spider['values'],
                                        percent = test_spider['percent'],
                                        ref_perc = test_spider['ref percent'],
                                        missing = test_spider['missing']
                                       ))


    labels = LabelSet(
        x='labs_x_cords', 
        y='labs_y_cords', 
        text='label_eth',
        text_align='center', 
        text_font ='helvetica',
        text_color = '#a0a0a0',
        source=source,
        render_mode='canvas'
    )

    hover = HoverTool(tooltips=[
            ("Ethnicity", "@label_eth"),
            ('Raw values', '@values'),
            ('Percent/%', "@percent{0.0}"),
            ('UK population percent/%', '@{ref_perc}{0.0}')
        ])

    q.patch(
        x= 'x_vals', 
        y='y_vals', 
        alpha = 0.35,
        line_alpha = 0,
        color = 'blue',
        source=source,
        legend_label = name
    )

    q.patch(
        x='x_ref', 
        y='y_ref',
        color = '#a0a0a0', 
        line_width = 0,
        alpha = 0.35,
        source = source, 
        legend_label ='UK Population Ratio')

    q.multi_line(
        'x_lines',
        'y_lines',
        source=source, 
        color="#a0a0a0", 
        line_width = 1
    )


    q.yaxis.major_label_text_font_size = '0pt'
    q.xaxis.major_label_text_font_size = '0pt'
    q.yaxis.axis_line_color = None
    q.xaxis.axis_line_color = None
    q.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    q.xaxis.minor_tick_line_color = None
    q.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    q.yaxis.minor_tick_line_color = None
    q.xgrid.grid_line_color = None
    q.ygrid.grid_line_color = None
    q.outline_line_width = 0
    q.background_fill_color = '#f5f5f5'
    q.background_fill_alpha = 0.9
    q.legend.location = 'top_left'
    q.title.text_color = '#a0a0a0'
    q.title.text_font_size = '24pt'
    q.title.text_font = "helvetica"
    q.legend.label_text_font = "helvetica"
    q.legend.label_text_color = "#a0a0a0"
    q.add_layout(labels)
    q.add_tools(hover)
    b = missing_plot(source,'missing','values')
    final_plot = gridplot([[q],[b]])
    return(final_plot)

if 'values' in test_eth.keys():
    eth_p = ethnicity_plot(test_eth,'',name)
else:
    var_list = [re.sub('values','',i) for i in test_eth if 'values' in i]
    tab_list = [Panel(child=ethnicity_plot(test_eth,i,name), title=re.sub(' ','',i)) for i in var_list]
    eth_p = Tabs(tabs = tab_list)

## Testing Gender

In [11]:
test_gender = graph_dict[name]['Gender']

In [12]:
source = ColumnDataSource(data = test_gender)

def gender_plot(source,addition,name):
    
    values = addition + 'values'
    ref_std = addition + 'reference standardised'
    missing_vals = addition + 'missing'
    percent = addition + 'percent'
    
    
    r = figure(
        x_range = list(source.data['Gender']), 
        title = 'Gender', 
        y_range=(0,max(source.data[values]) * 1.1 )
    )

    r.vbar(
        x = 'Gender',
        top = values, 
        width = 0.9, 
        color = 'blue',
        alpha = 0.35,
        line_alpha = 0,
        legend_label = name,    
        source = source
    )

    r.vbar(
        x = 'Gender',
        top = ref_std, 
        width = 0.885,
        fill_alpha = 0,
        line_color = '#a0a0a0', 
        line_width = 4,
        line_alpha = 1,
        legend_label = 'UK Population Ratio',
        source = source
    )


    hover2 = HoverTool(tooltips = [
        ('Gender range', '@gender'),
        ('Raw values', "@{"+values+"}"),
        ('Percent/%', "@{"+percent+"}{0.0}"),
        ('UK population percent/%', '@{ref percent}{0.0}')
        ],
               mode = 'mouse', name= 'data plot')

    r.yaxis.major_label_text_font_size = '0pt' 
    r.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    r.yaxis.minor_tick_line_color = None
    r.yaxis.axis_line_color = None
    r.xaxis.axis_line_color = None
    r.xaxis.major_tick_line_color = 'grey'
    r.xgrid.grid_line_color = None
    r.ygrid.grid_line_color = None
    r.outline_line_width = 0
    r.background_fill_color = '#f5f5f5'
    r.background_fill_alpha = 0.9
    r.legend.location = 'top_left'
    r.title.text_color = '#a0a0a0'
    r.title.text_font_size = '24pt'
    r.title.text_font = "helvetica"
    r.legend.label_text_font = "helvetica"
    r.legend.label_text_color = "#a0a0a0"
    r.add_tools(hover2)
    a = missing_plot(source,missing_vals,values)
    final_plot = gridplot([[r],[a]])
 
    return(final_plot)


if 'values' in source.data.keys():
    gender_p = gender_plot(source,'',name)
else:
    var_list = [re.sub('values','',i) for i in source.data.keys() if 'values' in i]
    tab_list = [Panel(child=gender_plot(source,i,name), title=re.sub(' ','',i)) for i in var_list]
    gender_p = Tabs(tabs = tab_list)

## Testing Socioeconomic status

In [13]:
test_ses = graph_dict[name]['Socioeconomic Status']

In [14]:
def ses_plot(data_dict,addition,name):
    
    imp_keys = ['Socioeconomic Status','ref percent'] + [k for k in data_dict.keys() if addition in k]
    
    test_spider = {re.sub(addition,'',k):v for k,v in data_dict.items() if k in imp_keys}

    num_vars = len(test_spider['values'])

    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    # rotate theta such that the first axis is at the top
    theta += np.pi/2

    def unit_poly_verts(theta):
        """Return vertices of polygon for subplot axes.
        This polygon is circumscribed by a unit circle centered at (0.5, 0.5)
        """
        x0, y0, r = [0.5] * 3
        verts = [(r*np.cos(t) + x0, r*np.sin(t) + y0) for t in theta]
        return verts

    def radar_patch(r, theta):
        yt = (r + 0.01) * np.sin(theta) + 0.5
        xt = (r + 0.01) * np.cos(theta) + 0.5
        return xt, yt

    verts = unit_poly_verts(theta)
    x = [i[0] for i in verts]
    y = [i[1] for i in verts]



    values = np.array(test_spider['values'])
    ref_std = np.array(test_spider['reference standardised'])
    missing_ses = test_spider['missing']
    missing_ses = missing_ses[0]/sum(values)

    values = values/(sum(values) * 2 )
    ref_std = ref_std/(sum(ref_std) * 2 )

    x_val,y_val = radar_patch(values, theta)
    x_ref,y_ref = radar_patch(ref_std,theta)

    label_ses = test_spider['Socioeconomic Status']

    x_lines = [[0.5,i] for i in x]
    y_lines = [[0.5,i] for i in y]
    new_line_max = np.array([max(values + ref_std)*0.55]*len(values))
    new_x,new_y = radar_patch(new_line_max,theta)
    new_x_lines = [[0.5,i] for i in new_x]
    new_y_lines = [[0.5,i] for i in new_y]

    s = figure(title = 'Socioeconomic Status',x_range=(min(new_x)-0.05,max(new_x)+0.05),y_range=(min(new_y)*0.9,max(new_y)*1.1))

    source = ColumnDataSource(data=dict(x_vals=x_val,
                                        y_vals=y_val,
                                        x_ref=x_ref,
                                        y_ref=y_ref,
                                        x_lines=new_x_lines,
                                        y_lines=new_y_lines,
                                        label_ses = label_ses,
                                        labs_x_cords = new_x,
                                        labs_y_cords = new_y,
                                        values = test_spider['values'],
                                        percent = test_spider['percent'],
                                        ref_perc = test_spider['ref percent'],
                                        missing = test_spider['missing']
                                       ))


    labels = LabelSet(
        x='labs_x_cords', 
        y='labs_y_cords', 
        text='label_ses',
        text_font ='helvetica',
        text_color = '#a0a0a0',
        source=source,
        render_mode='canvas',
        text_align='center'
    )

    hover = HoverTool(tooltips=[
            ("Socioeconomic", "@label_ses"),
            ('Raw values', '@values'),
            ('Percent/%', "@percent{0.0}"),
            ('UK population percent/%', '@{ref_perc}{0.0}')
        ])

    s.patch(
        x= 'x_vals', 
        y='y_vals', 
        alpha = 0.35,
        line_alpha = 0,
        color = 'blue',
        source=source,
        legend_label = 'UK Biobank'
    )

    s.patch(
        x='x_ref', 
        y='y_ref',
        color = '#a0a0a0', 
        alpha = 0.35,
        line_alpha = 0,
        source = source, 
        legend_label ='UK Population Ratio')

    s.multi_line(
        'x_lines',
        'y_lines',
        source=source, 
        color="#a0a0a0", 
        line_width = 1
    )


    s.yaxis.major_label_text_font_size = '0pt'
    s.xaxis.major_label_text_font_size = '0pt'
    s.yaxis.axis_line_color = None
    s.xaxis.axis_line_color = None
    s.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    s.xaxis.minor_tick_line_color = None
    s.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    s.yaxis.minor_tick_line_color = None
    s.xgrid.grid_line_color = None
    s.ygrid.grid_line_color = None
    s.outline_line_width = 0
    s.background_fill_color = '#f5f5f5'
    s.background_fill_alpha = 0.9
    s.legend.location = 'top_left'
    s.title.text_color = '#a0a0a0'
    s.title.text_font_size = '24pt'
    s.title.text_font = "helvetica"
    s.legend.label_text_font = "helvetica"
    s.legend.label_text_color = "#a0a0a0"
    s.add_layout(labels)
    s.add_tools(hover)
    b = missing_plot(source,'missing','values')
    final_plot = gridplot([[s],[b]])
    return(final_plot)

if 'values' in source.data.keys():
    ses_p = ses_plot(test_ses,'',name)
else:
    var_list = [re.sub('values','',i) for i in test_ses if 'values' in i]
    tab_list = [Panel(child=ses_plot(test_ses,i,name), title=re.sub(' ','',i)) for i in var_list]
    ses_p = Tabs(tabs = tab_list)

## Testing all together

In [15]:
out_plot = 'plots/UKB_test' + str(name) +'.html'
output_file(out_plot)
title = Div(text = """Data Representation Labels""",style={'font-size': '40pt', 'color': '#a0a0a0','font':'helvetica'}, 
            height=100)
t = gridplot([[age_p,eth_p],[gender_p,ses_p]])
final = gridplot([[title],[t]])
show(final) 